Code for fitting linear models 

In [1]:
%load_ext autoreload
%autoreload 2 

In [14]:
from pathlib import Path
import pickle
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from janelia_core.stats.regression import grouped_linear_regression_boot_strap
from janelia_core.stats.regression import grouped_linear_regression_boot_strap_stats
from janelia_core.utils.data_saving import append_ts

from keller_zlatic_vnc.linear_modeling import format_whole_brain_annots_table
from keller_zlatic_vnc.linear_modeling import one_hot_from_table

## Parameters go here

In [3]:
ps = {}
ps['data_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['data_file'] = r'dff_5_25_25_2019_10_22_10_58_11_107249.pkl'

# Specify variables that we predict from
ps['beh_before'] = ['Q', 'F', 'B']
ps['beh_after'] = ['Q', 'F', 'B']
ps['enc_beh_interactions'] = True
ps['enc_subjects'] = True
ps['closure'] = True # True if the only events we consider must start with a before_beh 
                     # behavior and end with an beh_after behavior
    
# How many bootstrap samples we use in each analysis
ps['n_bs_smps'] = 1000

ps['save_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'whole_brain_boot_strap'

## Load data

In [4]:
data_path = Path(ps['data_folder']) / Path(ps['data_file'])
with open(data_path, 'rb') as f:
    data = pickle.load(f)
event_annots = data['event_annots']

## Drop columns we don't need in the event annotation table

In [5]:
event_annots = event_annots.drop(['Manipulation Start', 
                                  'Manipulation End',
                                  'Time differ between end of PB from Stimulus ONSET',
                                  'Time differ between start of PB from Stimulus ONSET',
                                  'Time difference between start of SB from Stimus ONSET',
                                  'Interval Time',
                                  'Transtion Time'], axis='columns')

## Format the event annotation table so it is ready to be passed to one_hot_from_table

In [6]:
event_annots = format_whole_brain_annots_table(event_annots)

## Enforce closure if needed

In [7]:
if ps['closure']:
    print('Enforcing closure.')
    before_closure = np.asarray([b in set(ps['beh_before']) for b in event_annots['beh_before']], 
                                dtype=bool)
    after_closure = np.asarray([b in set(ps['beh_after']) for b in event_annots['beh_after']], 
                                dtype=bool)
    closure = np.logical_and(before_closure, after_closure)
    
    event_annots = event_annots[closure]

Enforcing closure.


## Get rid of events that have no behaviors of interest

In [8]:
before_ignore = np.asarray([b not in set(ps['beh_before']) for b in event_annots['beh_before']], 
                                dtype=bool)
after_ignore = np.asarray([b not in set(ps['beh_after']) for b in event_annots['beh_after']], 
                                dtype=bool)

ignore_rows = np.logical_and(before_ignore, after_ignore)

event_annots = event_annots[np.logical_not(ignore_rows)]

## Get groups of data (a group corresponds to each subject)

In [9]:
unique_ids = event_annots['subject_id'].unique()
g = np.zeros(len(event_annots))
for u_i, u_id in enumerate(unique_ids):
    g[event_annots['subject_id'] == u_id] = u_i

## Now get a one-hot encoding of variables we will fit too

In [10]:
one_hot_data, one_hot_vars = one_hot_from_table(table=event_annots, beh_before=ps['beh_before'], beh_after=ps['beh_after'], 
                                                enc_subjects=ps['enc_subjects'], 
                                                enc_beh_interactions=ps['enc_beh_interactions'])

# If we are not encoding subjects, we will include a mean in the regression, so we note that in the variable names
if not ps['enc_subjects']:
    one_hot_vars.append('mean')

## Now perform regression, with a bootstrap, for all supervoxels

In [11]:
dff_before = np.stack(event_annots['dff_before'].to_numpy())
dff_after = np.stack(event_annots['dff_after'].to_numpy())

n_supervoxels = dff_before.shape[1]

par_data_before = [(dff_before[:, n_i], one_hot_data, g, ps['n_bs_smps'], not ps['enc_subjects']) 
                   for n_i in range(n_supervoxels)]

par_data_after = [(dff_after[:, n_i], one_hot_data, g, ps['n_bs_smps'], not ps['enc_subjects']) 
                   for n_i in range(n_supervoxels)]

In [ ]:
t0 = time.time()

before_bs_rs = [grouped_linear_regression_boot_strap(*args) for args in par_data_before]

t1 = time.time()
print('Computaton time for ' + str(n_supervoxels) + ' super voxels: ' + str(t1 - t0))

In [ ]:
t0 = time.time()

after_bs_rs = [grouped_linear_regression_boot_strap(*args) for args in par_data_after]

t1 = time.time()
print('Computaton time for ' + str(n_supervoxels) + ' super voxels: ' + str(t1 - t0))

## Save results

In [ ]:
rs = {'ps': ps, 'one_hot_vars': one_hot_vars, 'before_bs_rs': before_bs_rs, 'after_bs_rs': after_bs_rs}

save_name = append_ts(ps['save_str']) + '.pkl'
save_path = Path(ps['save_folder']) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)